# 测试包含插件的浏览器会话

本笔记本演示了如何使用PlaywrightSessionPool获取和使用包含插件的浏览器会话。

In [1]:
headless = False  # 全局设置

In [2]:
import getpass
import os, sys, asyncio

print("Current user:", getpass.getuser())
print("UID:", os.getuid())
if sys.platform == "win32":
    # 设置 ProactorEventLoop 策略
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

sys.stderr.fileno = lambda: 2
sys.path.append(os.path.join(os.getcwd(), '..'))

Current user: minato_aqua
UID: 1000


In [3]:
import random
from app.models.RPA_browser.browser_info_model import UserBrowserInfoListParams, UserBrowserInfoCreateParams
from app.utils.depends.session_manager import DatabaseSessionManager
import sys
import asyncio

from app.services.RPA_browser.browser_db_service import BrowserDBService

if sys.platform == 'win32':
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

from app.services.RPA_browser.browser_session_pool.playwright_pool import get_default_session_pool
import uuid

In [4]:
# 创建或获取浏览器指纹
browser_token = uuid.UUID("0fc69198bccb4b00928b9372c99190b7")

async with DatabaseSessionManager.async_session() as session:
    browser_list = await BrowserDBService.list_fingerprint(UserBrowserInfoListParams(
        browser_token=browser_token,
        page=1,
        per_page=10
    ), session)

if browser_list.items:
    browser_id = browser_list.items[0].id
else:
    async with DatabaseSessionManager.async_session() as session:
        created_fingerprint = await BrowserDBService.create_fingerprint(UserBrowserInfoCreateParams(
            browser_token=browser_token,
            fingerprint_int=random.randint(0, 9999)
        ), session)
        browser_id = created_fingerprint.id

print(f"Using browser_token: {browser_token}")
print(f"Using browser_id: {browser_id}")

Using browser_token: 0fc69198-bccb-4b00-928b-9372c99190b7
Using browser_id: 7403833021427617792


In [5]:
# 使用PlaywrightSessionPool获取包含插件的浏览器会话
from app.models.RPA_browser.browser_session_model import SessionCreateParams

# 获取会话池
session_pool = get_default_session_pool()

# 创建会话参数
session_params = SessionCreateParams(
    browser_token=browser_token,
    browser_id=browser_id,
    headless=headless
)

# 获取包含插件的会话
plugined_session = await session_pool.get_session(session_params)
print(f"Got plugined session for browser_id: {browser_id}")

2025-12-10 15:35:38 | INFO        | app.services.RPA_browser.browser_session_pool.session_pool_model:reg_plugins:60 | [PLUGIN MANAGER] 🚀 开始注册插件 - 总配置数: 4
2025-12-10 15:35:38 | DEBUG       | app.services.RPA_browser.browser_session_pool.session_pool_model:reg_plugins:71 | [PLUGIN MANAGER] 🔧 正在注册插件: log - 默认日志插件
2025-12-10 15:35:38 | INFO        | app.services.site_rpa_operation.plugins.log_plugin:__init__:17 | [LOG PLUGIN] 📝 日志插件初始化 - 日志级别: INFO
2025-12-10 15:35:38 | INFO        | app.services.RPA_browser.browser_session_pool.session_pool_model:reg_plugins:80 | [PLUGIN MANAGER] ✅ 插件注册成功: log - 默认日志插件
2025-12-10 15:35:38 | DEBUG       | app.services.RPA_browser.browser_session_pool.session_pool_model:reg_plugins:71 | [PLUGIN MANAGER] 🔧 正在注册插件: page_limit - 默认页面限制插件
2025-12-10 15:35:38 | INFO        | app.services.site_rpa_operation.plugins.page_limit_plugin:__init__:17 | [PAGE LIMIT PLUGIN] 📄 页面限制插件初始化 - 最大页面数: 5
2025-12-10 15:35:38 | DEBUG       | app.services.site_rpa_operation.plugins

In [6]:
page = await plugined_session.get_current_page()

2025-12-10 15:37:03 | INFO        | app.services.RPA_browser.browser_session_pool.session_pool_model:__inject_plugins_to_page:239 | [PLUGIN INJECTION] 🎯 页面 134527770550480 插件注入完成 - 活跃插件数: 4
2025-12-10 15:37:03 | DEBUG       | app.services.RPA_browser.browser_session_pool.session_pool_model:__inject_plugins_to_page:240 | [PLUGIN INJECTION] 📋 已注入插件列表:
默认日志插件: 用户级别的默认日志记录插件
默认页面限制插件: 用户级别的页面数量限制插件
默认随机等待插件: 用户级别的操作间随机等待插件
默认重试插件: 用户级别的操作失败重试插件
2025-12-10 15:37:03 | DEBUG       | app.services.RPA_browser.browser_session_pool.session_pool_model:__inject_plugins_to_page:241 | [PLUGIN INJECTION] 📊 总增强页面数: 1
默认日志插件: 用户级别的默认日志记录插件
默认页面限制插件: 用户级别的页面数量限制插件
默认随机等待插件: 用户级别的操作间随机等待插件
默认重试插件: 用户级别的操作失败重试插件


下面可以写需要测试的脚本了，写完了丢到自定义的类里就行

In [13]:
await page.goto("https://live.bilibili.com/")
await page.click(".header-login-entry")
await page.fill(".bili-mini-account input", 'xxx')
await page.fill('.bili-mini-password input', 'xxx')
await page.click(".universal-btn.login-btn")

2025-12-10 15:42:21 | INFO        | app.services.site_rpa_operation.plugins.log_plugin:_log_start_operation:50 | [LOG PLUGIN] 🚀 开始执行操作 - 时间戳: 1765352541.87
2025-12-10 15:42:21 | DEBUG       | app.services.site_rpa_operation.plugins.log_plugin:_log_start_operation:51 | [LOG PLUGIN] 📋 配置信息 - 日志级别: INFO, 插件描述: 用户级别的默认日志记录插件
2025-12-10 15:42:21 | DEBUG       | app.services.site_rpa_operation.plugins.log_plugin:_log_operation_context:56 | [LOG PLUGIN] 📋 操作上下文 - 浏览器引擎: BaseUndetectedPlaywright
2025-12-10 15:42:21 | DEBUG       | app.services.site_rpa_operation.plugins.log_plugin:_log_operation_context:57 | [LOG PLUGIN] 📋 操作上下文 - 会话状态: 已连接
2025-12-10 15:42:21 | INFO        | app.services.site_rpa_operation.plugins.page_limit_plugin:_check_page_limit:33 | [PAGE LIMIT PLUGIN] 📊 当前页面数量: 1/5
2025-12-10 15:42:21 | DEBUG       | app.services.site_rpa_operation.plugins.page_limit_plugin:_check_page_limit:34 | [PAGE LIMIT PLUGIN] 📋 页面使用率: 20.0%
2025-12-10 15:42:21 | INFO        | app.services.site_rp

In [11]:
a = await page.locator(".geetest_item_wrap:last-of-type").evaluate(r"""el=>{
                                const style = window.getComputedStyle(el);
                                const bg = style.backgroundImage;
                                const match = bg.match(/url\(["']?(.*?)["']?\)/);
                                return match ? match[1] : null;
                                }""")
print(a)
print(page.is_closed())

None
False


In [8]:
from app.services.geetest.captcha_break import acb

geetest_url = await page.locator(".geetest_item_wrap:last-of-type").evaluate(r"""el=>{
                                const style = window.getComputedStyle(el);
                                const bg = style.backgroundImage;
                                const match = bg.match(/url\(["']?(.*?)["']?\)/);
                                return match ? match[1] : null;
                                }""")
print(f"geetest_url:{geetest_url}")
result = await acb.predict_chinese_click_from_url(url=geetest_url)

2025-12-10 15:39:07 | INFO        | app.services.geetest.captcha_break:_find_model_file:88 | 找到yolo模型文件: /home/minato_aqua/.cache/modelscope/hub/models/Amorter/CaptchaBreakerModels/yolov11n_captcha.onnx
2025-12-10 15:39:07 | INFO        | app.services.geetest.captcha_break:_find_model_file:88 | 找到siamese模型文件: /home/minato_aqua/.cache/modelscope/hub/models/Amorter/CaptchaBreakerModels/siamese.onnx


/home/minato_aqua/bili-fastapi-browser-rpa/.venv/lib/python3.13/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:123: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


2025-12-10 15:39:08 | INFO        | app.services.geetest.captcha_break:_load_models:60 | 模型加载成功，设备: cuda
geetest_url:None


In [ ]:
bound = await page.locator(".geetest_item_wrap:last-of-type").bounding_box()
for x, y in result:
    print(x, y)
    print(bound)
    click_position = {"x": x / 360 * bound.get('width'), "y": y / 360 * bound.get('height')}
    print(click_position)
    await asyncio.sleep(1)
    await page.locator(".geetest_table_box").click(position=click_position)
await page.screenshot(path='./geetest.png')


In [ ]:
await page.locator(".geetest_commit_tip").click()

In [15]:
await page.screenshot(path='./login.png')

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x07\x80\x00\x00\x048\x08\x02\x00\x00\x00g\xb1V\x14\x00\x00\x10\x00IDATx\x9c\xec\xbd\t\x83\xdc\xb6\x95\xae\r\x90\xb5\xf4\xde\xadVk\xb5-\xdb\x89\x93\x99$\xd7\x99{o\xfe\xffO\x98\xf9&\x93\xdc\xc4N\xe28\x96d[[\xb7z\xef\xdaH|/\x00\x12\x04I\x90\x04Y\xacRK:O\x9cV\x15\x89\x8d \x08\x02/N\x1d\x0c\xfe\xf0\x87?0\x82 \x08\x82 \x08\x82 \x08\x82 \x08\x82 \x08\x82\xe8\x9b\x01#\x08\x82 \x08\x82 \x08\x82 \x08\x82 \x08\x82 \x88\x15@\x024A\x10\x04A\x10\x04A\x10\x04A\x10\x04A\x10\x04\xb1\x12H\x80&\x08\x82 \x08\x82 \x08\x82 \x08\x82 \x08\x82 V\x02\t\xd0\x04A\x10\x04A\x10\x04A\x10\x04A\x10\x04A\x10\xc4J \x01\x9a \x08\x82 \x08\x82 \x08\x82 \x08\x82 \x08\x82X\t$@\x13\x04A\x10\x04A\x10\x04A\x10\x04A\x10\x04A\x10+\x81\x04h\x82 \x08\x82 \x08\x82 \x08\x82 \x08\x82 \x08b%\x90\x00M\x10\x04A\x10\x04A\x10\x04A\x10\x04A\x10\x04A\xac\x04\x12\xa0\t\x82 \x08\x82 \x08\x82 \x08\x82 \x08\x82 \x88\x95@\x024A\x10\x04A\x10\x04A\x10\x04A\x10\x04A\x10\x04\xb1\x12H\x80&\x08\x82 \x08\x82

In [ ]:
# await page.goto("https://bot-detector.rebrowser.net/")
# await page.screenshot(path='./1.png',full_page=True)